In [5]:

"""
Data Fetcher Script - Investment Portfolio Analysis
Fetches stock data from Yahoo Finance API and stores in MySQL database
Author: Data Analyst Portfolio Project
"""
import yfinance as yf
import mysql.connector
import pandas as pd
from datetime import datetime, timedelta
import time
import logging
from typing import List, Dict, Any
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)
class StockDataFetcher:
    def __init__(self, db_config: Dict[str, str]):
        """Initialize with database configuration"""
        self.db_config = db_config
        self.connection = None
        
    def connect_to_db(self):
        """Establish database connection"""
        try:
            self.connection = mysql.connector.connect(**self.db_config)
            logger.info("Successfully connected to MySQL database")
            return True
        except mysql.connector.Error as e:
            logger.error(f"Error connecting to MySQL: {e}")
            return False
    
    def close_connection(self):
        """Close database connection"""
        if self.connection:
            self.connection.close()
            logger.info("Database connection closed")
    
    def get_stock_symbols(self) -> List[str]:
        """Get list of stock symbols from database"""
        try:
            cursor = self.connection.cursor()
            cursor.execute("SELECT symbol FROM stocks")
            symbols = [row[0] for row in cursor.fetchall()]
            cursor.close()
            return symbols
        except mysql.connector.Error as e:
            logger.error(f"Error fetching stock symbols: {e}")
            return []
    
    def fetch_stock_data(self, symbol: str, period: str = "6mo") -> pd.DataFrame:
        """Fetch stock data from Yahoo Finance API"""
        try:
            logger.info(f"Fetching data for {symbol}...")
            
            # Create yfinance ticker object
            ticker = yf.Ticker(symbol)
            
            # Get historical data
            hist_data = ticker.history(period=period)
            
            if hist_data.empty:
                logger.warning(f"No data found for {symbol}")
                return pd.DataFrame()
            
            # Reset index to get date as column
            hist_data.reset_index(inplace=True)
            
            #1 Add symbol column
            hist_data['Symbol'] = symbol
            
            # Check available columns and handle missing ones
            logger.info(f"Available columns for {symbol}: {list(hist_data.columns)}")
            
            # Rename columns to match database schema
            column_mapping = {
                'Date': 'date',
                'Open': 'open_price',
                'High': 'high_price',
                'Low': 'low_price',
                'Close': 'close_price',
                'Volume': 'volume',
                'Symbol': 'symbol'
            }
            
            # Handle Adj Close column (may not always be present)
            if 'Adj Close' in hist_data.columns:
                column_mapping['Adj Close'] = 'adj_close'
            else:
                # Use Close price as adj_close if Adj Close is not available
                hist_data['Adj Close'] = hist_data['Close']
                column_mapping['Adj Close'] = 'adj_close'
                logger.warning(f"Adj Close not found for {symbol}, using Close price")
            
            hist_data.rename(columns=column_mapping, inplace=True)
            
            # Convert date to string format
            hist_data['date'] = hist_data['date'].dt.strftime('%Y-%m-%d')
            
            logger.info(f"Successfully fetched {len(hist_data)} records for {symbol}")
            return hist_data
            
        except Exception as e:
            logger.error(f"Error fetching data for {symbol}: {e}")
            return pd.DataFrame()
    
    def store_stock_data(self, df: pd.DataFrame):
        """Store stock data in MySQL database"""
        if df.empty:
            logger.warning("No data to store")
            return
        
        try:
            cursor = self.connection.cursor()
            
            # Prepare INSERT statement with ON DUPLICATE KEY UPDATE
            insert_query = """
                INSERT INTO stock_prices 
                (symbol, date, open_price, high_price, low_price, close_price, volume, adj_close)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                open_price = VALUES(open_price),
                high_price = VALUES(high_price),
                low_price = VALUES(low_price),
                close_price = VALUES(close_price),
                volume = VALUES(volume),
                adj_close = VALUES(adj_close)
            """
            
            # Convert DataFrame to list of tuples for insertion
            data_tuples = []
            for _, row in df.iterrows():
                try:
                    data_tuples.append((
                        row['symbol'],
                        row['date'],
                        float(row['open_price']) if pd.notna(row['open_price']) else None,
                        float(row['high_price']) if pd.notna(row['high_price']) else None,
                        float(row['low_price']) if pd.notna(row['low_price']) else None,
                        float(row['close_price']) if pd.notna(row['close_price']) else None,
                        int(row['volume']) if pd.notna(row['volume']) else None,
                        float(row['adj_close']) if pd.notna(row['adj_close']) else None
                    ))
                except KeyError as e:
                    logger.error(f"Missing column {e} in data for {row.get('symbol', 'unknown')}")
                    logger.info(f"Available columns: {list(df.columns)}")
                    raise
            
            # Execute batch insert
            cursor.executemany(insert_query, data_tuples)
            self.connection.commit()
            
            logger.info(f"Successfully stored {len(data_tuples)} records for {df['symbol'].iloc[0]}")
            cursor.close()
            
        except mysql.connector.Error as e:
            logger.error(f"Error storing data: {e}")
            self.connection.rollback()
    
    def check_for_price_alerts(self, symbol: str):
        """Check for 10% weekly price changes and generate alerts"""
        try:
            cursor = self.connection.cursor()
            
            # Get last 7 days of data
            query = """
                SELECT date, close_price 
                FROM stock_prices 
                WHERE symbol = %s 
                ORDER BY date DESC 
                LIMIT 7
            """
            cursor.execute(query, (symbol,))
            results = cursor.fetchall()
            
            if len(results) < 2:
                cursor.close()
                return
            
            latest_price = float(results[0][1])
            week_ago_price = float(results[-1][1])
            
            # Calculate percentage change
            price_change = ((latest_price - week_ago_price) / week_ago_price) * 100
            
            # Check if change is >= 10%
            if abs(price_change) >= 10:
                alert_type = 'rise' if price_change > 0 else 'fall'
                
                # Check if alert already exists for today
                today = datetime.now().strftime('%Y-%m-%d')
                check_query = """
                    SELECT id FROM price_alerts 
                    WHERE symbol = %s AND alert_date = %s
                """
                cursor.execute(check_query, (symbol, today))
                
                if not cursor.fetchone():
                    # Insert new alert
                    insert_alert = """
                        INSERT INTO price_alerts 
                        (symbol, alert_type, percentage_change, price_from, price_to, alert_date)
                        VALUES (%s, %s, %s, %s, %s, %s)
                    """
                    cursor.execute(insert_alert, (
                        symbol, alert_type, price_change, 
                        week_ago_price, latest_price, today
                    ))
                    self.connection.commit()
                    
                    logger.info(f"Alert generated for {symbol}: {price_change:.2f}% change")
            
            cursor.close()
            
        except mysql.connector.Error as e:
            logger.error(f"Error checking price alerts for {symbol}: {e}")
    
    def update_all_stocks(self):
        """Fetch and store data for all stocks in database"""
        if not self.connect_to_db():
            return
        
        try:
            symbols = self.get_stock_symbols()
            logger.info(f"Updating data for {len(symbols)} stocks...")
            
            for symbol in symbols:
                # Fetch data from API
                stock_data = self.fetch_stock_data(symbol)
                
                if not stock_data.empty:
                    # Store in database
                    self.store_stock_data(stock_data)
                    
                    # Check for alerts
                    self.check_for_price_alerts(symbol)
                
                # Rate limiting - be respectful to Yahoo Finance API
                time.sleep(1)
            
            logger.info("Data update completed successfully")
            
        except Exception as e:
            logger.error(f"Error during data update: {e}")
        
        finally:
            self.close_connection()
def main():
    """Main execution function"""
    # Database configuration
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': 'root',  # Add your MySQL password here
        'database': 'portfolio_analysis'
    }
    
    # Create fetcher instance
    fetcher = StockDataFetcher(db_config)
    
    # Update all stock data
    fetcher.update_all_stocks()
if __name__ == "__main__":
    main()


2025-09-26 04:05:36,169 - INFO - Successfully connected to MySQL database
2025-09-26 04:05:36,172 - INFO - Updating data for 8 stocks...
2025-09-26 04:05:36,172 - INFO - Fetching data for AAPL...
2025-09-26 04:05:36,405 - INFO - Available columns for AAPL: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Symbol']
2025-09-26 04:05:36,406 - WARNING - Adj Close not found for AAPL, using Close price
2025-09-26 04:05:36,408 - INFO - Successfully fetched 127 records for AAPL
2025-09-26 04:05:36,431 - INFO - Successfully stored 127 records for AAPL
2025-09-26 04:05:37,433 - INFO - Fetching data for AMZN...
2025-09-26 04:05:37,765 - INFO - Available columns for AMZN: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Symbol']
2025-09-26 04:05:37,765 - WARNING - Adj Close not found for AMZN, using Close price
2025-09-26 04:05:37,768 - INFO - Successfully fetched 127 records for AMZN
2025-09-26 04:05:37,790 - INFO - Successfully store